# import

In [32]:
import time
from datetime import datetime
import os
import gpxpy
import gpxpy.gpx
import math
import numbers
import geoutils

## Haversine fomula를 이용하여 거리를 구해보자

In [33]:
#euclidean.py
import numbers
import math

class GeoUtil:
    """
    Geographical Utils
    """
    @staticmethod
    def degree2radius(degree):
        return degree * (math.pi/180)
    
    @staticmethod
    def get_harversion_distance(x1, y1, x2, y2, round_decimal_digits=5):
        """
        경위도 (x1,y1)과 (x2,y2) 점의 거리를 반환
        Harversion Formula 이용하여 2개의 경위도간 거래를 구함(단위:Km)
        """
        if x1 is None or y1 is None or x2 is None or y2 is None:
            return None
        assert isinstance(x1, numbers.Number) and -180 <= x1 and x1 <= 180
        assert isinstance(y1, numbers.Number) and  -90 <= y1 and y1 <=  90
        assert isinstance(x2, numbers.Number) and -180 <= x2 and x2 <= 180
        assert isinstance(y2, numbers.Number) and  -90 <= y2 and y2 <=  90

        R = 6371 # 지구의 반경(단위: km)
        dLon = GeoUtil.degree2radius(x2-x1)    
        dLat = GeoUtil.degree2radius(y2-y1)

        a = math.sin(dLat/2) * math.sin(dLat/2) \
            + (math.cos(GeoUtil.degree2radius(y1)) \
              *math.cos(GeoUtil.degree2radius(y2)) \
              *math.sin(dLon/2) * math.sin(dLon/2))
        b = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        return round(R * b, round_decimal_digits)

    @staticmethod
    def get_euclidean_distance(x1, y1, x2, y2, round_decimal_digits=5):        
        """
        유클리안 Formula 이용하여 (x1,y1)과 (x2,y2) 점의 거리를 반환
        """
        if x1 is None or y1 is None or x2 is None or y2 is None:
            return None
        assert isinstance(x1, numbers.Number) and -180 <= x1 and x1 <= 180
        assert isinstance(y1, numbers.Number) and  -90 <= y1 and y1 <=  90
        assert isinstance(x2, numbers.Number) and -180 <= x2 and x2 <= 180
        assert isinstance(y2, numbers.Number) and  -90 <= y2 and y2 <=  90

        dLon = abs(x2-x1) # 경도 차이
        if dLon >= 180:   # 반대편으로 갈 수 있는 경우
            dLon -= 360   # 반대편 각을 구한다
        dLat = y2-y1      # 위도 차이
        return round(math.sqrt(pow(dLon,2)+pow(dLat,2)),round_decimal_digits)

# 거리구하는 함수

논문에서 말하길 이동창의 거리가 작기 떄문에 피타고라스 공식을 사용해도 괜찮다고 나옴

In [34]:
def distance(x1, y1, x2, y2):
    distance = math.sqrt((x1-x2)**2 + (y1-y2)**2)
    #GeoUtil.get_harversion_distance(x1, y1, x2, y2)
    return distance

# 그냥 코드 옆에 폴더 안의 모든 gpx파일 경로를 구해오는 함수

In [35]:
def readDir(): 
    DIR = os.getcwd()                #절대 경로 get
    files = []                       #파일 목록을 반환할 리스트
    dirnames = os.listdir(DIR)       #프로그램과 같은 폴더에 있는 폴더를 구함(데이터들이 들어있는 폴더 리스트)
    for dirname in dirnames:     
        DIR = os.path.abspath(dirname)
        if os.path.isdir(dirname):
            for filename in os.listdir(dirname):
                if(filename[-4:] == ".gpx"):
                    files.append(os.path.join(DIR, filename))    # 디렉토리 절대 경로 + 파일 이름 = 파일의 절대 경로
    files.sort()
    return files

# 파일 넣으면 해당 파일에 있는 gpx데이터 리스트 반환

In [36]:
def parse(fileName):
    gpxes = []
    gpx_file = open(fileName, "r", encoding = "utf-8") #파일 오픈
    gpx = gpxpy.parse(gpx_file) #gpx파싱 시작
    for track in gpx.tracks:    
        for segment in track.segments:
            for point in segment. points: #포인트 즉 필요한 데이터 객체로 들어옴
                timestr = point.time.strftime("%Y-%m-%d %H:%M:%S")
                date = datetime.strptime(timestr, "%Y-%m-%d %H:%M:%S")
                timestamp = time.mktime(date.timetuple())
                temp = {}
                temp["datetime"] = str(date)
                temp["timestamp"] = timestamp
                temp["latitude"] = point.latitude
                temp["longitude"] = point.longitude
                gpxes.append(temp)   #gpxes 리스트에 시간,위도,경도 정보를 저장.
    return gpxes  # 추출한 gpx정보 리턴

In [37]:
def get_data_from_files(files):
    data = []
    #f = open('test.txt', mode='wt', encoding='utf-8')
    for file in files:      # gpx파일들을 한개씩 뽑아서 사용
        gpxes = parse(file) # 파싱해서  gpx 시간,위도,경도 정보 추출
        for gpx in gpxes:   # gpx 정보를 사용한다.
            data.append(gpx)
    return data

# gpx 데이터 가공
### 각각의 속도filtered 열 첨가
corrected 값은 바뀔 예정이지만 형식의 용이를 위해 origin과 같은 값으로 미리 삽입해 놓는다

In [38]:
#{'datetime': '2019-08-16 09:00:31', 'timestamp': 1565913631.0, 'latitude': 37.55016333333333, 'longitude': 126.92437333333334}, {'datetime': '2019-08-16 09:00:32', 'timestamp': 1565913632.0, 'latitude': 37.55007, 'longitude': 126.92435833333333},

In [48]:
def append_velocity(data):
    #0번째 데이터에는 속도 0으로 넣을것
    data_len=len(data)
    data[0]['originalV']=0 #측정값 속력
    data[0]['correctedV']=0 #교정값 속력
    data[0]['Vlongti']=0 #longtitude  속력
    data[0]['Vlati']=0 #latitude  속력 
    data[0]['filtered']='false' #가공 되야하는 친구(가공 되야 하면 true)
    data[0]['accelFiltered']='false' #속도가공 되야하는 친구(가공 되야 하면 true)
    data[0]['retrieved']='false' # 속력 교정 not 가속도 교정
    data[0]['interpolated']='false' # 속력 교정 not 가속도 교정
    data[0]['a']=0 #가속도
    data[0]['latitudeCorrected']= data[0]['latitude']
    data[0]['longitudeCorrected']= data[0]['longitude']
    

    for param in range(1,data_len):
        
        if data[param-1]["timestamp"] < data[param]["timestamp"]:
            t=data[param]["timestamp"]- data[param-1]["timestamp"]
            
            data[param]['filtered']='false'
            data[param]['accelFiltered']='false'
            data[param]['retrieved']='false'
            data[param]['interpolated']='false' # 속력 교정 not 가속도 교정
            data[param]['latitudeCorrected']= data[param]['latitude']
            data[param]['longitudeCorrected']= data[param]['longitude']
            data[param]['originalV']=distance(data[param]["latitude"], data[param]["longitude"], data[param-1]["latitude"], data[param-1]["longitude"])/ t
            data[param]['Vlongti']= abs(data[param]["longitude"]-data[param-1]["longitude"])/t
            data[param]['Vlati']=abs(data[param]["latitude"]-data[param-1]["latitude"])/t
            data[param]['correctedV']=data[param]['originalV']
            if(param)==1:
                data[param]['a']=0
            else:
                data[param]['a']=(data[param]['originalV']-data[param-1]['originalV'])/t
        else:
                data[param]['originalV']=data[param-1]['originalV']
                data[param]['correctedV']=data[param-1]['correctedV']
                data[param]['Vlongti']=data[param-1]['Vlongti']
                data[param]['Vlati']=data[param-1]['Vlati']
                data[param]['filtered']=data[param-1]['filtered']
                data[param]['accelFiltered']=data[param-1]['accelFiltered']
                data[param]['retrieved']=data[param-1]['retrieved']
                data[param]['interpolated']=data[param-1]['interpolated']
                data[param]['a']=data[param-1]['a']
                data[param]['latitudeCorrected']= data[param-1]['latitudeCorrected']
                data[param]['longitudeCorrected']= data[param-1]['longitudeCorrected']
    
            
    return data

In [49]:
ex=[{'datetime': '2019-08-16 09:00:31', 'timestamp': 1565913631.0, 'latitude': 37.55016333333333, 'longitude': 126.92437333333334}, {'datetime': '2019-08-16 09:00:32', 'timestamp': 1565913632.0, 'latitude': 37.55007, 'longitude': 126.92435833333333}]

In [50]:
append_velocity(ex)

[{'datetime': '2019-08-16 09:00:31',
  'timestamp': 1565913631.0,
  'latitude': 37.55016333333333,
  'longitude': 126.92437333333334,
  'originalV': 0,
  'correctedV': 0,
  'Vlongti': 0,
  'Vlati': 0,
  'filtered': 'false',
  'accelFiltered': 'false',
  'retrieved': 'false',
  'interpolated': 'false',
  'a': 0,
  'latitudeCorrected': 37.55016333333333,
  'longitudeCorrected': 126.92437333333334},
 {'datetime': '2019-08-16 09:00:32',
  'timestamp': 1565913632.0,
  'latitude': 37.55007,
  'longitude': 126.92435833333333,
  'filtered': 'false',
  'accelFiltered': 'false',
  'retrieved': 'false',
  'interpolated': 'false',
  'latitudeCorrected': 37.55007,
  'longitudeCorrected': 126.92435833333333,
  'originalV': 9.453100608285356e-05,
  'Vlongti': 1.5000000004761205e-05,
  'Vlati': 9.333333333216842e-05,
  'correctedV': 9.453100608285356e-05,
  'a': 0}]

## sign함수

sign(x) 에서 x 가 0 보다 크면 1, 0이면 0, 0보다 작으면 -1 반환

In [51]:
def sign(x):
    if x>0:
        return 1
    elif x==0:
        return 0
    else:
        return -1
    

# 구현 그 자체

In [52]:
def publish(data):

    #Initial Window Size 
    IWS = 5
    
    #Maximum Window Size, IWS <= MWS
    MWS = 10
    n = IWS
    
    #user SL s
    SL=0.95 
    
    #moving average
    MA = 0
    
    #moving standard deviation
    MSD = 0
    
    #min speed
    MINSPEED = 2.00
    #max acceleration
    MAXaccel=10.8
    
    #error tolerance of distantce
    ET = 999
    
    #loop
    i = 0 
    
    #Number of Consecutive Errors
    NCE = 0 # 오류 정정 횟수
    MA = 0
    MAlati=0
    MAlongti=0
    MSD = 0
    
    
    #MSD, MA , MAlati, MAlongti caculate
    for i in range(len(data)-1):
        if data[i]["timestamp"] < data[i+1]["timestamp"]:     # 시간이 다음께 더 커야하니까
            MINSPEED = min(ET/(data[i+1]["timestamp"] - data[i]["timestamp"]), MINSPEED)
            if i > n:
                MAlati=abs(data[i-n-1]["latitude"]-data[i]["latitude"])/n
                MAlongti=abs(data[i-n-1]["longitude"]-data[i]["longitude"])/n
                
                MA = distance(data[i-n-1]["latitude"], data[i-n-1]["longitude"], data[i]["latitude"], data[i]["longitude"]) / n
                MSDlati_tmp=0
                MSDlongti_tmp=0
                for param in range(i-n-1,i+1):
                    MSDlongti_tmp= MSDlongti_tmp+((data[param]["longitude"]-MAlongti)**2)/n
                for param in range(i-n-1,i+1):
                    MSDlati_tmp= MSDlati_tmp+((data[param]["latitude"]-MAlati)**2)/n
                
                MSDlati=math.sqrt(MSDlati_tmp)
                MSDlongti=   math.sqrt(MSDlongti_tmp)
                MSD =math.sqrt(MSDlati**2+MSDlongti**2)
            else:
                pass
    # calculate LP(Location Parameter)
    LP=MA-MSD
    
    
    Vori=distance(data[i-1]["latitude"], data[i-1]["longitude"], data[i]["latitude"], data[i]["longitude"]) / (data[i-1]["timestamp"]- data[i]["timestamp"])
    
    
    
    
    #caculate Verr( Compute Maximum possible velocity in user SL s )#유저 s 가 가능한 최대 속도 계산
    molecular=1-SL # 분자
    try:
        param=1/MSD
    except:
        param=0
        
    denominator=math.exp(-param)
    Verr   = ( molecular/denominator )+LP
    # Compute velocity for checking whether Vi +1is to be calibrated (교정될지에 대해 확인)
    Vcalib=(0.005/ math.exp(-param) )+LP
    
    #오류 탐지 ==> 이동창 크기 조정
    if((data[i+1]["originalV"]>Verr) or (data[i+1]["a"]>=MAXaccel))and (data[i+1]["originalV"]>MINSPEED):
            data[i+1]["filtered"]='true'
            NCE=NCE+1
            if(n+1>MWS):
                n=MWS
            else:
                 n=n+1  # Window Size Adjustment-Increase 
    else:
        NCE=0
        if(n-1<IWS):
                n=IWS
        else:
                 n=n-1  # Window Size Adjustment-Decrease
    #오류 탐지
    if (data[i+1]["originalV"]>=Vcalib) and (data[i+1]["originalV"]> MINSPEED):
        data[i+1]["correctedV"]= Vcalib # Calibration of Speed
    if(data[i+1]["a"]>=MAXaccel):
        data[i+1]['accelFiltered']='true'
        data[i+1]["a"]=MAXaccel
        data[i+1]["correctedV"]=MA
        time_between_10=data[i+1]['timestamp']-data[i]['timestamp']
        data[i+1]['latitudeCorrected']=data[i+1]['latitude']+sign(data[i+1]['latitude']-data[i]['latitude'])*MAlati*time_between
        data[i+1]['longitudeCorrected']=data[i+1]['longitude']+sign(data[i+1]['longitude']-data[i]['longitude'])*MAlongti*time_between
                
    if(data[i]["originalV"]<=Verr)and(data[i]["filtered"]=="True")and(data[i]["accelFiltered"]=="False"):
        data[i]['retrieved']='True'
        data[i]['correctedV']=data[i]['originalV']
        data[i]['latitudeCorrected']= data[i]['latitude']
        data[i]['longitudeCorrected']= data[i]['longitude']# Backtracking: Look back one step and Restore with original values 
        NCE=0
        if(n-1<IWS):
                n=IWS
        else:
                 n=n-1  # Window Size Adjustment-Decrease
    if(data[i]["filtered"]=="True")or(data[i]["accelFiltered"]=="True"):
        time_between_11=data[i+1]['timestamp']-data[i-1]['timestamp']
        time_between_01=data[i]['timestamp']-data[i-1]['timestamp']
        data[i]['correctedV']=(data[i+1]['correctedV']-data[i-1]['correctedV'])*time_between_01/time_between_11+data[i-1]['correctedV']
        # Estimation of speed
        data[i]['latitudeCorrected']= (data[i+1]['latitudeCorrected']-data[i-1]['latitudeCorrected'])*time_between_01/time_between_11+data[i-1]['latitudeCorrected']
        data[i]['longitudeCorrected']= (data[i+1]['longitudeCorrected']-data[i-1]['longitudeCorrected'])*time_between_01/time_between_11+data[i-1]['longitudeCorrected']
        # Estimation of Position
        data[i]['interpolated']='true'
    
    return data
    
        
        
#             data[0]['originalV']=0 #측정값 속력
#     data[0]['correctedV']=0 #교정값 속력
#     data[0]['Vlongti']=0 #longtitude  속력
#     data[0]['Vlati']=0 #latitude  속력 
#     data[0]['filtered']='false' #가공 되야하는 친구(가공 되야 하면 true)
#     data[0]['accelFiltered']='false' #속도가공 되야하는 친구(가공 되야 하면 true)
#     data[0]['retrieved']='false' # 속력 교정 not 가속도 교정
#     data[0]['a']=0 #가속도
#     data[0]['latitudeCorrected']= data[0]['latitude']
#     data[0]['longitudeCorrected']= data[0]['longitude']

                
           
    
            
            
    

In [53]:
import math
MSD=2
parm=2
math.exp(-1/MSD)

0.6065306597126334

In [54]:
1<=2

True

# main

In [55]:
def main():
    try:
        data = []
        files = readDir() #file list 반환
        data = get_data_from_files(files)
        data = append_velocity(data)
        data =publish(data)
        print(data)
    except ValueError:
        print(ValueError)
        
        

In [56]:
if __name__ == "__main__":
    main()
    

[{'datetime': '2019-08-16 09:00:31', 'timestamp': 1565913631.0, 'latitude': 37.55016333333333, 'longitude': 126.92437333333334, 'originalV': 0, 'correctedV': 0, 'Vlongti': 0, 'Vlati': 0, 'filtered': 'false', 'accelFiltered': 'false', 'retrieved': 'false', 'interpolated': 'false', 'a': 0, 'latitudeCorrected': 37.55016333333333, 'longitudeCorrected': 126.92437333333334}, {'datetime': '2019-08-16 09:00:32', 'timestamp': 1565913632.0, 'latitude': 37.55007, 'longitude': 126.92435833333333, 'filtered': 'false', 'accelFiltered': 'false', 'retrieved': 'false', 'interpolated': 'false', 'latitudeCorrected': 37.55007, 'longitudeCorrected': 126.92435833333333, 'originalV': 9.453100608285356e-05, 'Vlongti': 1.5000000004761205e-05, 'Vlati': 9.333333333216842e-05, 'correctedV': 9.453100608285356e-05, 'a': 0}, {'datetime': '2019-08-16 09:00:35', 'timestamp': 1565913635.0, 'latitude': 37.55021333333333, 'longitude': 126.92428166666667, 'filtered': 'false', 'accelFiltered': 'false', 'retrieved': 'false'

# raw code 

# 하버사이오

Hofmann-WellenhofBernhardandLichteneggerHerbertandWasleElmar.GNSS–globalnavigation satellitesystems:GPS,GLONASS,Galileo,andmore. Springer;2007

# 위도 경도 msd 설정참고 깃허브

https://github.com/NMullally/SoapBoxTest/blob/6569aac805d61843327a24338eb5ee876331858f/SoapBoxTest/FilterData.cpp